# Analisis de Calidad de Datos de la Hoja de Trabajo 'HECHOS' 

Importamos las librerias

In [29]:
import numpy as np
import pandas as pd
import pyproj
#from openpyxl import load_workbook   

---
<br/><br/>

## Cargamos el libro homicidios.xlsx

In [30]:
ruta_excel = r'D:\PEDRO\AHORA\A LA CARGA!!\WORKING AREA\Cursos and Tuts\Henry\Labs\PI ANALYTICS\DESARROLLO DEL PI\data\homicidios.xlsx'
xlsx_homicidios = pd.ExcelFile(ruta_excel)


Mostramos la estructuras del libro homicidios.xlsx

In [31]:
nombres_hojas = xlsx_homicidios.sheet_names
for nombre in nombres_hojas:
    df = xlsx_homicidios.parse(sheet_name=nombre)
    size_bytes = df.memory_usage(deep=True).sum() 
    print(f"Hoja de trabajo: {nombre}")
    print(f"Cantidad de filas: {len(df)}")
    print(f"Cantidad de columnas: {len(df.columns)}")
    print(f"Tamaño en bytes: {round(size_bytes / 1024, 1)} Kbs")
    print("\n")

Hoja de trabajo: HECHOS
Cantidad de filas: 696
Cantidad de columnas: 21
Tamaño en bytes: 685.7 Kbs


Hoja de trabajo: DICCIONARIO_HECHOS
Cantidad de filas: 46
Cantidad de columnas: 3
Tamaño en bytes: 11.8 Kbs


Hoja de trabajo: VICTIMAS
Cantidad de filas: 717
Cantidad de columnas: 10
Tamaño en bytes: 272.0 Kbs


Hoja de trabajo: DICCIONARIO_VICTIMAS
Cantidad de filas: 21
Cantidad de columnas: 3
Tamaño en bytes: 5.3 Kbs


Hoja de trabajo: clas
Cantidad de filas: 28
Cantidad de columnas: 7
Tamaño en bytes: 10.0 Kbs




---
<br/><br/>

## Inspeccion Del DataFrame:

To DF

In [32]:
df_homicidios_hechos = pd.read_excel(ruta_excel, sheet_name='HECHOS')

In [33]:
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


---
<br/><br/>

## Analisis de Nulos

### Analisis

Computamos las columnas con valores faltantes

In [34]:
# Calcular el conteo de valores nulos por columna
conteo_valores_nulos = df_homicidios_hechos.isnull().sum()

# Filtra las columnas con nulos mayores a  1 y ordena descendentemente
columnas_con_nulos = conteo_valores_nulos[df_homicidios_hechos.isnull().sum() > 0].sort_values(ascending = False)

print('Registros totales: ', df_homicidios_hechos.shape[0])
print("\nColumnas con valores nulos:")
print(columnas_con_nulos)

Registros totales:  696

Columnas con valores nulos:
Altura                   567
Cruce                    171
Dirección Normalizada      8
Calle                      1
dtype: int64


El DF tiene 696 registros y 4 campos con datos faltantes: 'Altura', 'Cruce', 'Dirección Normalizada' y 'Calle'

Los campos con datos faltantes parecen estar relacionados al campo 'LUGAR_DEL_HECHO'  
Para estudiarlos mejor pintamos una muestra de los campos en cuestion

In [35]:
# Muestra los nombres de los campos
print('columnas en df_homicidios_hechos: ','[' + ', '.join(df_homicidios_hechos.keys()) + ']') 

columnas en df_homicidios_hechos:  [ID, N_VICTIMAS, FECHA, AAAA, MM, DD, HORA, HH, LUGAR_DEL_HECHO, TIPO_DE_CALLE, Calle, Altura, Cruce, Dirección Normalizada, COMUNA, XY (CABA), pos x, pos y, PARTICIPANTES, VICTIMA, ACUSADO]


In [36]:
#Pinta una muestra de 25 filas entre  'LUGAR_DEL_HECHO', y 'Cruce'
df_homicidios_hechos.sample(n=10, random_state= 40)[['LUGAR_DEL_HECHO','Calle','Cruce', 'Dirección Normalizada']]
#df_homicidios_hechos.sample(n=25, random_state= 50).loc[:, 'LUGAR_DEL_HECHO':'Cruce']

,LUGAR_DEL_HECHO,Calle,Cruce,Dirección Normalizada
643,AU FRONDIZI Y SUAREZ AV.,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y S...
132,AV MARTIN GARCIA 300,"GARCIA, MARTIN AV.",NaN,"GARCIA, MARTIN AV. 300"
508,"BRIN, MINISTRO Y SUAREZ","BRIN, MINISTRO",SUAREZ,"BRIN, MINISTRO y SUAREZ"
556,"SCALABRINI ORTIZ, RAUL AV. Y PARAGUAY","SCALABRINI ORTIZ, RAUL AV.",PARAGUAY,"SCALABRINI ORTIZ, RAUL AV. y PARAGUAY"
668,"CHILAVERT, MARTINIANO, CORONEL Y VARELA AV.","CHILAVERT, MARTINIANO, CORONEL",VARELA AV.,"CHILAVERT, MARTINIANO, CORONEL y VARELA AV."
46,AV GAONA Y ESPINOSA,GAONA AV.,ESPINOSA,GAONA AV. y ESPINOSA
683,"PAZ, GRAL. AV. Y EZEIZA","PAZ, GRAL. AV.",EZEIZA,"PAZ, GRAL. AV. y EZEIZA"
413,Santander 1974,SANTANDER,NaN,SANTANDER 1974
212,24 DE NOVIEMBRE 390,24 DE NOVIEMBRE,NaN,24 DE NOVIEMBRE 390
669,SAN LUIS 2730,SAN LUIS,NaN,SAN LUIS 2730


Comprobamos que tanto 'Calle', 'Cruce' y 'Dirección Normalizada' se crean a partir de la columna 'LUGAR_DEL_HECHO'.   

Ahora revisamos la columna 'Dirección Normalizada', que tiene 8 datos faltantes: 

In [37]:
normalizados_faltantes = df_homicidios_hechos[df_homicidios_hechos['Dirección Normalizada'].isnull()][['LUGAR_DEL_HECHO','Calle','Dirección Normalizada']]
normalizados_faltantes

,LUGAR_DEL_HECHO,Calle,Dirección Normalizada
38,AUTOPISTA LUGONES PK 10000,"LUGONES, LEOPOLDO AV.",NaN
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA BUENOS AIRES - LA PLATA,NaN
119,SD,NaN,NaN
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA PERITO MORENO,NaN
181,AU DELLEPIANE 2400,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN
313,AUTOPISTA LUGONES KM 4.7,"LUGONES, LEOPOLDO AV.",NaN
546,"LUGONES, LEOPOLDO AV. KM 6,1","LUGONES, LEOPOLDO AV.",NaN
621,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA BUENOS AIRES - LA PLATA,NaN


Se infiere que los datos faltantes en 'Dirección Normalizada' no se normalizaron por que los siniestros ocurrieron en autopistas   Asimismo se muestra el dato faltante en 'Calle' de indice 119 coincidiendo con uno de 'Dirección Normalizada'  

In [38]:
# Captura la fila con indice 119
registro_indice_119 = df_homicidios_hechos.iloc[119]

# Transpone para mostrarlo en horizontal
registro_indice_119.to_frame().T 

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


El registro 119 no tiene datos significativos. 

### Conclusiones y Acciones a Tomar
* Los datos faltantes en 'Altura' representan una enorme proporción del conjunto de datos totales: SE ELIMINA
* El campo 'Cruce' tiene 171 valores faltantes y poco peso significativo por si mismo: SE ELIMINA
* Los datos de 'Dirección Normalizada' son muy similares a los del campo 'Calle': SE IMPUTAN LOS 8 DATOS FALTANTES POR EL CAMPO 'Calle'
* El unico dato faltante de 'Calle' se encuentra en un registro sin datos significativos: SE ELIMINA Y SE RESETEA EL INDICE

Eliminacion de 'Altura' y 'Cruce'

In [39]:
df_homicidios_hechos = df_homicidios_hechos.drop(columns=['Altura'])
df_homicidios_hechos = df_homicidios_hechos.drop(columns=['Cruce'])

Imputacion de 'Dirección Normalizada'

In [40]:
# Reemplaza todos los valores nulos en 'Dirección Normalizada' con los valores de 'Calle'
df_homicidios_hechos['Dirección Normalizada'].fillna(df_homicidios_hechos['Calle'], inplace=True)
print("Se imputaron todos los datos los datos faltantes en 'Dirección Normalizada'")
#Checkamos los cambios
df_homicidios_hechos[df_homicidios_hechos['Dirección Normalizada'] == df_homicidios_hechos['Calle']][['LUGAR_DEL_HECHO','Calle','Dirección Normalizada']]

Se imputaron todos los datos los datos faltantes en 'Dirección Normalizada'


,LUGAR_DEL_HECHO,Calle,Dirección Normalizada
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI
38,AUTOPISTA LUGONES PK 10000,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV."
71,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA BUENOS AIRES - LA PLATA,AUTOPISTA BUENOS AIRES - LA PLATA
139,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO
176,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV."
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO
181,AU DELLEPIANE 2400,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI
313,AUTOPISTA LUGONES KM 4.7,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV."


Eliminacion del indice 119 y reseteo del index

In [41]:
df_homicidios_hechos = df_homicidios_hechos.drop(119)
print("Datos faltantes en 'Calle':")
print(df_homicidios_hechos[['Calle']].isnull().sum())

df_homicidios_hechos.reset_index(drop=True, inplace=True)

Datos faltantes en 'Calle':
Calle    0
dtype: int64


Confirmamos que no haya datos faltantes en df_homicidios_hechos

In [42]:
print('Datos faltantes:')
print(df_homicidios_hechos.isnull().sum())

Datos faltantes:
ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Dirección Normalizada    0
COMUNA                   0
XY (CABA)                0
pos x                    0
pos y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
dtype: int64


---
<br/><br/>

## Tratamiento de Valores Invalidos

###  'Numero de Victimas'  
Verificamos si el campo 'Nro de Victimas' tiene valores invalidos

In [43]:
# Crea un nuevo DataFrame con las filas que cumplen con la condición
df_valores_diferentes = df_homicidios_hechos[~df_homicidios_hechos['N_VICTIMAS'].isin([1, 2, 3])]

# Muestra el nuevo DataFrame
df_valores_diferentes

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


In [44]:
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  Calle                  695 non-null    object        
 11  Dirección Normalizada  695 non-null    object        
 12  COMUNA                 695 non-null    int64         
 13  XY (C

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


---
<br/><br/>

###  'HORA'  
Verificamos si 'HORA' tiene valores invalidos

In [45]:
# Define un patrón de expresión regular para el formato deseado (HH:MM:SS)
patron_formato = r'^\d{2}:\d{2}:\d{2}'

# Utiliza str.match para encontrar las filas que coinciden con el patrón
valores_no_validos = df_homicidios_hechos[~df_homicidios_hechos['HORA'].astype(str).str.match(patron_formato)]

# Muestra las filas que no tienen el formato deseado
valores_no_validos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
438,2019-0022,1,2019-03-15,2019,3,15,1900-01-02 00:00:00,0,ENTRE RIOS AV. BAJO PUENTE AUTOPISTA (1260 APROX),AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 1260,1,Point (106608.39392805 100635.27735528),-58.39123782,-34.62352245,AUTO-SD,AUTO,SD
440,2019-0024,1,2019-03-19,2019,3,19,1900-01-14 07:12:00,7,ANTARTIDA ARGENTINA AV. 1325,AVENIDA,ANTARTIDA ARGENTINA AV.,ANTARTIDA ARGENTINA AV. 1325,1,Point (108368.93688513 104860.74716674),-58.37208118,-34.58541957,MOTO-CARGAS,MOTO,CARGAS
495,2019-0080,1,2019-09-17,2019,9,17,1900-01-04 18:40:00,18,"CABILDO AV. y PAZ, GRAL. AV.",GRAL PAZ,CABILDO AV.,"CABILDO AV. y PAZ, GRAL. AV.",13,Point (98817.74986778 110096.31367495),-58.47617802,-34.53825652,PEATON-MOTO,PEATON,MOTO
517,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Tenemos 4 valores invalidos en 'HORA'. Los imputamos con el valor mas frecuente de 'HORA' y modificamos la columna 'HH' con el valor pertinente

In [46]:
indices_a_modificar = [438, 440, 495, 517]

# Encuentra el valor más repetido en 'Hora Exacta'
hora_mas_frecuente = df_homicidios_hechos['HORA'].mode()[0]
print(f'Valor más frecuente en "HORA": {hora_mas_frecuente}')

for indice in indices_a_modificar:    
    # Modifica los registros seleccionados
    print(f"Valor del indice {indice} antes de la actualización:", df_homicidios_hechos.at[indice, 'HORA'])
    df_homicidios_hechos.at[indice, 'HORA'] = hora_mas_frecuente
    print(f"Valor del indice {indice} después de la actualización:", df_homicidios_hechos.at[indice, 'HORA'])

    # Actualizar la columna 'HH' en los índices especificados
    hora_mas_frecuente_string = df_homicidios_hechos['HORA'].mode()[0].strftime('%H:%M:%S')
    df_homicidios_hechos.at[indice, 'HH'] = int(hora_mas_frecuente_string.split(':')[0])


# Verificar que los cambios se hayan aplicado correctamente
df_homicidios_hechos.loc[indices_a_modificar]

Valor más frecuente en "HORA": 14:00:00
Valor del indice 438 antes de la actualización: 1900-01-02 00:00:00
Valor del indice 438 después de la actualización: 14:00:00
Valor del indice 440 antes de la actualización: 1900-01-14 07:12:00
Valor del indice 440 después de la actualización: 14:00:00
Valor del indice 495 antes de la actualización: 1900-01-04 18:40:00
Valor del indice 495 después de la actualización: 14:00:00
Valor del indice 517 antes de la actualización: SD
Valor del indice 517 después de la actualización: 14:00:00


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
438,2019-0022,1,2019-03-15,2019,3,15,14:00:00,14,ENTRE RIOS AV. BAJO PUENTE AUTOPISTA (1260 APROX),AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 1260,1,Point (106608.39392805 100635.27735528),-58.39123782,-34.62352245,AUTO-SD,AUTO,SD
440,2019-0024,1,2019-03-19,2019,3,19,14:00:00,14,ANTARTIDA ARGENTINA AV. 1325,AVENIDA,ANTARTIDA ARGENTINA AV.,ANTARTIDA ARGENTINA AV. 1325,1,Point (108368.93688513 104860.74716674),-58.37208118,-34.58541957,MOTO-CARGAS,MOTO,CARGAS
495,2019-0080,1,2019-09-17,2019,9,17,14:00:00,14,"CABILDO AV. y PAZ, GRAL. AV.",GRAL PAZ,CABILDO AV.,"CABILDO AV. y PAZ, GRAL. AV.",13,Point (98817.74986778 110096.31367495),-58.47617802,-34.53825652,PEATON-MOTO,PEATON,MOTO
517,2019-0103,1,2019-12-18,2019,12,18,14:00:00,14,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


---
<br/><br/>

### 'PARTICIPANTES' y sus campos dependientes
'PARTICIPANTES' tiene varios valores invalidos tipo 'MOTO-SD', 'SD-AUTO', 'SD-SD'.  
LOS CAMPOS 'VICTIMA' y 'ACUSADO' herendan parcialmente estos valores invalidos. Lo tratamos de la siguiente manera:
* Imputamos los valores que contienen 'SD' del campo 'VICTIMA' con su valor mas frecuente. 
* Imputamos los valores que contienen 'SD' del campo 'ACUSADO' con su valor mas frecuente.
* Imputamos los valotres que contienen 'SD' del campo 'PARTICIPANTES'con los valores de 'VICTIMA' y 'ACUSADO'

Checkamos si hay variantes en minuscula de 'SD' y mostramos los filas con valores invalidos

In [47]:
# Filtrar las filas que contienen las cadenas mencionadas en las columnas 'VICTIMA' y 'ACUSADO'
# El argumento case=False considerará automáticamente todas las combinaciones de mayúsculas y minúsculas: 'sd', 'SD', 'sD', 'Sd'
filtro = df_homicidios_hechos['VICTIMA'].str.contains('sd', case=False) | df_homicidios_hechos['ACUSADO'].str.contains('sd', case=False)

# Aplicar el filtro al DataFrame
df_valores_invalidos = df_homicidios_hechos[filtro][['PARTICIPANTES','VICTIMA', 'ACUSADO']]

# Captura los indices de las filas con valores invalidosa
indices_valores_invalidos = df_valores_invalidos.index.tolist()
print(len(df_valores_invalidos))

df_valores_invalidos


26


,PARTICIPANTES,VICTIMA,ACUSADO
3,MOTO-SD,MOTO,SD
32,MOTO-SD,MOTO,SD
35,SD-SD,SD,SD
38,MOTO-SD,MOTO,SD
57,AUTO-SD,AUTO,SD
76,SD-SD,SD,SD
83,PEATON-SD,PEATON,SD
88,SD-CARGAS,SD,CARGAS
92,SD-SD,SD,SD
105,PEATON-SD,PEATON,SD


No vemos variantes en minusculas de 'SD', ahora imputamos

In [48]:
# Reemplazar 'SD' en 'VICTIMA' con el valor más frecuente en 'VICTIMA'
#valor_mas_frecuente_victima = df_homicidios_hechos['VICTIMA'].mode().iloc[0]
#df_homicidios_hechos['VICTIMA'] = df_homicidios_hechos['VICTIMA'].replace('SD', valor_mas_frecuente_victima)

# Reemplazar 'SD' en 'ACUSADO' con el valor más frecuente en 'ACUSADO'
#valor_mas_frecuente_acusado = df_homicidios_hechos['ACUSADO'].mode().iloc[0]
#df_homicidios_hechos['ACUSADO'] = df_homicidios_hechos['ACUSADO'].replace('SD', valor_mas_frecuente_acusado)

# Concatenar 'VICTIMA' y 'ACUSADO' y asignar el resultado a 'PARTICIPANTES'
#df_homicidios_hechos['PARTICIPANTES'] = df_homicidios_hechos['VICTIMA'] + '-' + df_homicidios_hechos['ACUSADO']

# Captura el valor más frecuente de 'VICTIMA' y 'ACUSADO'
valor_mas_frecuente_victima = df_homicidios_hechos['VICTIMA'].mode().iloc[0]
valor_mas_frecuente_acusado = df_homicidios_hechos['ACUSADO'].mode().iloc[0]

for indice in indices_valores_invalidos:

    # Imputa cada valor no valido en 'VICTIMA' y 'ACUSADO' con el valor masfrecuente
    df_homicidios_hechos.at[indice, 'VICTIMA'] = valor_mas_frecuente_victima
    df_homicidios_hechos.at[indice, 'ACUSADO'] = valor_mas_frecuente_acusado

    # Imputa cada valor no valido en 'PARTICIPANTES'
    valor_victima = df_homicidios_hechos.at[indice, 'VICTIMA']
    valor_acusado = df_homicidios_hechos.at[indice, 'ACUSADO']
    valor_concatenado = f"{valor_victima}-{valor_acusado}"
    df_homicidios_hechos.at[indice, 'PARTICIPANTES'] = valor_concatenado

df_homicidios_hechos.loc[indices_valores_invalidos][['PARTICIPANTES','VICTIMA','ACUSADO']]

,PARTICIPANTES,VICTIMA,ACUSADO
3,MOTO-AUTO,MOTO,AUTO
32,MOTO-AUTO,MOTO,AUTO
35,MOTO-AUTO,MOTO,AUTO
38,MOTO-AUTO,MOTO,AUTO
57,MOTO-AUTO,MOTO,AUTO
76,MOTO-AUTO,MOTO,AUTO
83,MOTO-AUTO,MOTO,AUTO
88,MOTO-AUTO,MOTO,AUTO
92,MOTO-AUTO,MOTO,AUTO
105,MOTO-AUTO,MOTO,AUTO


In [49]:
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  Calle                  695 non-null    object        
 11  Dirección Normalizada  695 non-null    object        
 12  COMUNA                 695 non-null    int64         
 13  XY (C

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


---
<br/><br/>

## Manejo de valores Atipicos

### Coordenadas Geograficas 'Longitud Geografica' y 'Latitud Geografica'
Reemplazamos los valores con un  '.' a nan 

In [50]:

# Reemplaza los valores que contienen '.' con NaN en 'pos x' y 'pos y'
df_homicidios_hechos['pos x'].replace('.', np.nan, inplace=True)
df_homicidios_hechos['pos y'].replace('.', np.nan, inplace=True)

# Muestra los valores nan en 'pos x','pos y'
coordenadas_faltantes = df_homicidios_hechos[df_homicidios_hechos['pos x'].isnull()][['ID','pos x','pos y']]
#print(df[['pos x','pos y']].dtypes)
coordenadas_faltantes



,ID,pos x,pos y
38,2016-0052,NaN,NaN
106,2016-0136,NaN,NaN
138,2016-0174,NaN,NaN
175,2017-0042,NaN,NaN
179,2017-0050,NaN,NaN
180,2017-0051,NaN,NaN
255,2017-0140,NaN,NaN
312,2018-0039,NaN,NaN
545,2020-0026,NaN,NaN
558,2020-0039,NaN,NaN


---
<br/><br/>

## Guardamos lo Trabajado

In [51]:
# Almacenamos una copia para la etapa de Data Transformation
df_homicidios_hechos.to_csv('./data/df_homicidios_hechos_cleaned.csv', index=False)

## Conversion de Tipos de Datos

### NO FUNCIONA BIEN

Los campos de coordenadas geograficas son de tipo object, para ser usados deben ser convertidos a tipo float  
Primero probamos los cambios en una copia 'df' de prueba

In [52]:
# Convertir las columnas 'Latitud Geográfica' y 'Longitud Geográfica' a tipo float
#df = df_homicidios_hechos.copy()
#df['pos x'] = df['pos x'].astype(float)
#df_homicidios_hechos['pos x'] = df_homicidios_hechos['pos x'].astype(float)

#df['pos x'] = pd.to_numeric(df['pos x'], errors='coerce')
#df['pos y'] = pd.to_numeric(df['pos y'], errors='coerce')             

Verificamos si se crearon nulos

In [53]:
'''
# Calcular el conteo de valores nulos por columna
conteo_valores_nulos = df.isnull().sum()

# Filtra las columnas con nulos mayores a  1 y ordena descendentemente
columnas_con_nulos = conteo_valores_nulos[df.isnull().sum() > 0].sort_values(ascending = False)

print("\nColumnas con valores nulos:")
print(columnas_con_nulos)
'''

'\n# Calcular el conteo de valores nulos por columna\nconteo_valores_nulos = df.isnull().sum()\n\n# Filtra las columnas con nulos mayores a  1 y ordena descendentemente\ncolumnas_con_nulos = conteo_valores_nulos[df.isnull().sum() > 0].sort_values(ascending = False)\n\nprint("\nColumnas con valores nulos:")\nprint(columnas_con_nulos)\n'

Se crearon 11 datos nulos en cada coordenada, checkamos esas filas en el dataframe original 'df_homicidios_hechos'

In [54]:
'''
coordenadas_faltantes = df[df['pos x'].isnull()][['LUGAR_DEL_HECHO','pos x','pos y']]
print(df[['pos x','pos y']].dtypes)
coordenadas_faltantes
'''

"\ncoordenadas_faltantes = df[df['pos x'].isnull()][['LUGAR_DEL_HECHO','pos x','pos y']]\nprint(df[['pos x','pos y']].dtypes)\ncoordenadas_faltantes\n"

In [55]:
'''
indices = [38, 106, 138, 175, 179, 180, 255, 312, 545, 558, 620]

# Filtrar las filas con los índices especificados
filas_seleccionadas = df_homicidios_hechos.loc[indices][['LUGAR_DEL_HECHO','pos x','pos y']]

# Mostrar las filas seleccionadas
filas_seleccionadas
'''

"\nindices = [38, 106, 138, 175, 179, 180, 255, 312, 545, 558, 620]\n\n# Filtrar las filas con los índices especificados\nfilas_seleccionadas = df_homicidios_hechos.loc[indices][['LUGAR_DEL_HECHO','pos x','pos y']]\n\n# Mostrar las filas seleccionadas\nfilas_seleccionadas\n"

Esos puntos no sirven en el EDA, actualizamos df_homicidios_hechos 

df_homicidios_hechos = df

In [56]:
'''
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)
'''

'\n# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes\ndf_homicidios_hechos.info()\ndf_homicidios_hechos.head(2)\n'

---
<br/><br/>